In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from util import traffic_Signal

# Load the pre-trained models.
background_object = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
emergency_model = load_model('config/emergency_identification.h5')

# Load the first video.
video1 = cv2.VideoCapture('dataset/carsvid.wmv')

# Load the second video.
video2 = cv2.VideoCapture('dataset/carsvid.wmv')

while True:
    # Read a new frame from the first video.
    detected_cars = set()
    ret1, frame1 = video1.read()

    # Read a new frame from the second video.
    ret2, frame2 = video2.read()

    # Check if frames are not read correctly.
    if not ret1 or not ret2:
        # Break the loop.
        break

    # Apply the background object on the frame to get the segmented mask for the first video.
    fg_mask1 = background_object.apply(frame1)

    # Perform thresholding to get rid of the shadows.
    _, fg_mask1 = cv2.threshold(fg_mask1, 250, 255, cv2.THRESH_BINARY)

    # Apply some morphological operations to make sure you have a good mask.
    fg_mask1 = cv2.erode(fg_mask1, None, iterations=1)
    fg_mask1 = cv2.dilate(fg_mask1, None, iterations=2)

    # Detect contours in the frame.
    contours1, _ = cv2.findContours(fg_mask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a copy of the frame to draw bounding boxes around the detected vehicles for the first video.
    frame_copy1 = frame1.copy()

    # Loop over each contour found in the frame.
    for cnt1 in contours1:
        # Make sure the contour area is somewhat higher than some threshold to make sure it's a vehicle and not some noise.
        if cv2.contourArea(cnt1) > 400:
            car_id = tuple(cv2.boundingRect(cnt1))
            # Retrieve the bounding box coordinates from the contour.
            if car_id not in detected_cars:
                # Add the car to the set of detected cars.
                detected_cars.add(car_id)

                # Retrieve the bounding box coordinates from the contour.
                x, y, width, height = car_id

                # Draw a bounding box around the car.
                cv2.rectangle(frame_copy1, (x, y), (x + width, y + height), (0, 0, 255), 2)

                # Write Car Detected near the bounding box drawn.
                cv2.putText(frame_copy1, f'Car Detected {len(detected_cars)}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3,
                            (0, 255, 0), 1, cv2.LINE_AA)

    # Apply the background object on the frame to get the segmented mask for the second video.
    fg_mask2 = background_object.apply(frame2)

    # Perform thresholding to get rid of the shadows.
    _, fg_mask2 = cv2.threshold(fg_mask2, 250, 255, cv2.THRESH_BINARY)

    # Apply some morphological operations to make sure you have a good mask.
    fg_mask2 = cv2.erode(fg_mask2, None, iterations=1)
    fg_mask2 = cv2.dilate(fg_mask2, None, iterations=2)

    # Detect contours in the frame.
    contours2, _ = cv2.findContours(fg_mask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a copy of the frame to draw bounding boxes around the detected vehicles for the second video.
    frame_copy2 = frame2.copy()

    # Loop over each contour found in the frame.
    for cnt2 in contours2:
        # Make sure the contour area is somewhat higher than some threshold to make sure it's a vehicle and not some noise.
        if cv2.contourArea(cnt2) > 400:
            # Retrieve the bounding box coordinates from the contour.
            x2, y2, width2, height2 = cv2.boundingRect(cnt2)

            # Extract the region of interest (ROI) for the emergency vehicle classification.
            roi2 = frame2[y2:y2 + height2, x2:x2 + width2]

            # Resize the ROI to match the expected input shape of the model.
            input_roi2 = cv2.resize(roi2, (350, 350)) / 255.0  # Adjust the size based on your model's input size

            # Perform prediction using the loaded emergency identification model.
            prediction2 = emergency_model.predict(np.expand_dims(input_roi2, axis=0))

            # Assuming the model outputs a probability score, you can set a threshold for identification.
            threshold2 = 0.95
            if prediction2[0] > threshold2:
                cv2.rectangle(frame_copy2, (x2, y2), (x2 + width2, y2 + height2), (0, 0, 255), 2)
                cv2.putText(frame_copy2, 'Emergency Vehicle Detected', (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (0, 255, 0), 2, cv2.LINE_AA)
            else:
                cv2.rectangle(frame_copy2, (x2, y2), (x2 + width2, y2 + height2), (255, 0, 0), 2)
                cv2.putText(frame_copy2, 'Non-Emergency Vehicle Detected', (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (0, 255, 0), 2, cv2.LINE_AA)

    # Resize both frames to have the same height.
    common_height = min(frame_copy1.shape[0], frame_copy2.shape[0])
    frame_copy1 = cv2.resize(frame_copy1, (int(frame_copy1.shape[1] * common_height / frame_copy1.shape[0]), common_height))
    frame_copy2 = cv2.resize(frame_copy2, (int(frame_copy2.shape[1] * common_height / frame_copy2.shape[0]), common_height))

    # Stack the frames side by side for display.
    stacked_frames = np.hstack((frame_copy1, frame_copy2))

    # Display the stacked frames.
    cv2.imshow('Vehicle and Emergency Identification', cv2.resize(stacked_frames, None, fx=0.5, fy=0.5))

    # Wait until a key is pressed.
    k = cv2.waitKey(50) & 0xFF

    # Check if 'q' key is pressed.
    if k == ord('q'):
        # Break the loop.
        break

print(f'The Count of the vehicles is: {len(detected_cars)}')
        
# Release the VideoCapture objects.
video1.release()
video2.release()

# Close the window.
cv2.destroyAllWindows()




1/1 [==============================] - 0s 233ms/step


1/1 [==============================] - 0s 281ms/step
The Count of the vehicles is: 8


In [2]:
#Calculating the Seconds for Green signal
count = len(detected_cars)
seconds = min(count +(count/2)+10 , 90);
int(seconds)

19

In [3]:
traffic_Signal.traffic_signal(1,int(seconds))

Dynamic Signal Switching Phase

OPENING LANE-1: 
----------------------------------------------------------------------------------
Lane 1                Lane 2                Lane 3                Lane 4
  ⚪                   🔴                    🔴                   🔴
  ⚪                   ⚪                    ⚪                   ⚪
  🟢                   ⚪                    ⚪                   ⚪

----------------------------------------------------------------------------------

LANE-1 OPENED !

 Calculating Signal Open-Close Timing...

LANE-1 will CLOSE after 19 seconds ...................----------------------------------------------------------------------------------

CLOSING LANE-1: 
----------------------------------------------------------------------------------

Lane 1                Lane 2                Lane 3                Lane 4
  🔴                   🔴                    🔴                   🔴
  ⚪                   ⚪                    ⚪                   ⚪
  ⚪           